# Ce jupyter s'appuie sur un paper :
https://hal.telecom-paris.fr/hal-02367908/document
## Il a été réalisé en collaboration avec Frederic Haykal 
donc une partie du code sera commune à nos deux rendus.

# Idée maitresse de la solution :
Nous allons produire un modèle par base.\
Ce modèle va apprendre à produire la distance entre la base et les messages reçus par cette base en s'appuyant sur le rssi.\
Le modèle va modéliser les perturbations terrain afin de pouvoir prédire une distance avec un rssi non fiable.

### Explication de l'idée
Cas du GPS : le rssi donne la puissance recu du signal. ainsi avec cet valeur on peut définir la distance parcouru par le signal.\
Notre cas : Mais par construction dans notre cas ce rssi est perturbé par autre chose que la distance (le terrain). Nous utiliserons donc du Machine Learning pour apprendre ces perturbations et arriver a calculer cette distance.

### de plus
On remarque que si l'on met les distances au lieu des 1 dans la df_feature on a de très bon résultat.\
Notre objectif sera donc de créer cette matrice dans le jeu de test en s'appuyant sur un modèle appris sur le train.

Un dernier model prédira la position du message en s'appuyant sur cette nouvelle matrice.\
NB : nous rajouterons également les barycentres venant des autres jupyter pour encore augmenter la précision.

Comme on peut le voir à la fin, malgré un overfit non négligeable les résultats sont prometteurs

In [1]:
import pandas as pd
import numpy as np
from IotTools import *
#from IpyTools import *
pd.options.mode.chained_assignment = None  # default='warn'
import pickle
from vincenty import vincenty

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import VotingRegressor

from sklearn.model_selection import cross_val_predict

In [2]:
df_mess_train = pd.read_csv('mess_train_list.csv')
df_mess_test = pd.read_csv('mess_test_list.csv')
pos_train = pd.read_csv('pos_train_list.csv')
listOfBs = np.union1d(df_mess_train.bsid.unique(),df_mess_test.bsid.unique())

In [3]:
size = len(listOfBs)
size

259

# On corrige les bases outliers

In [4]:
X_train= Correct_Bases (df_mess_train)
X_train[X_train.bs_lat>64].shape[0];

Nous avons 27 bases outliers
Base 9949 non vu
il reste 0 base avec lat >60


In [5]:
X_test= Correct_Bases (df_mess_test)
X_test[X_test.bs_lat>64].shape[0];

Nous avons 23 bases outliers
Correction manuelle de la bsid 9949
il reste 0 base avec lat >60


# On créer la liste des coordonnées de toutes les bases

In [6]:
#On crééer une matrice contenant les coordonnées des bases
base_coord = pd.concat([X_train,X_test]).drop_duplicates(subset='bsid')[['bsid','bs_lat','bs_lng']].set_index('bsid')

# On créer une feature distance

In [7]:
msg_coord = (pos_train.set_index(keys=df_mess_train.messid.values).reset_index()).groupby('index').mean()
X_train['dist']=0
for i in X_train.index:
    bs_Coord = (X_train.iloc[i].bs_lat,X_train.iloc[i].bs_lng)
    msg_Coord = (msg_coord.loc[X_train.iloc[i].messid].lat,msg_coord.loc[X_train.iloc[i].messid].lng)
    X_train.loc[i,'dist']=vincenty(bs_Coord,msg_Coord)
X_train.head(1)

,messid,bsid,did,nseq,rssi,time_ux,bs_lat,bs_lng,dist
0,573bf1d9864fce1a9af8c5c9,2841,473335.0,0.5,-121.5,1.463546e+12,39.617794,-104.954917,1.270478


# La fct feat matrice propre a notre cas où l'on fait apparaitre le Rssi

In [8]:
def feat_mat_const(df, listOfBs):
    """On a rajouté une colonne contenant le nombre de détection par msg
    cela servira à identifié la pondération de la prédiction
    """
    df_group = df.groupby("messid", as_index=False)
    nb_mess = len(df.messid.unique())
    df_feat = pd.DataFrame(np.zeros((nb_mess, len(listOfBs))), columns=listOfBs, index=df.messid.unique())
    id_list = [0] * nb_mess

    for i, (key, elmt) in enumerate(df_group):
        df_feat.loc[key,elmt.bsid]= elmt.rssi.values #ici on met le rssi au lieu des 1 
        id_list[i] = key
        
    """Rajout des BaryCentres"""
            
    a =df.rssi.values
    df['rssi_reshape'] = (10**(a/10))
    df['bs_lat_pond'] = df.bs_lat * df.rssi_reshape
    df['bs_lng_pond'] = df.bs_lng * df.rssi_reshape
    BCW_lat = df.groupby('messid').bs_lat_pond.sum()/df.groupby('messid').rssi_reshape.sum()
    BCW_lat.name='BCW_lat'
    BCW_lng = df.groupby('messid').bs_lng_pond.sum()/df.groupby('messid').rssi_reshape.sum()
    BCW_lng.name='BCW_lng'

    #Pour le barycentre ajout min , max + mean 

    BC_lat = df.groupby('messid').agg(['mean', 'min', 'max'])[['bs_lat']]
    BC_lat.columns = BC_lat.columns.droplevel()

    BC_lng = df.groupby('messid').agg(['mean', 'min', 'max'])[['bs_lng']]
    BC_lng.columns = BC_lng.columns.droplevel()

    dev_lat =df.groupby(['did','messid']).agg(['mean', 'min', 'max'])[['bs_lat']].reset_index()
    dev_lat.columns = dev_lat.columns.droplevel()
    dev_lat.columns= ['did','messid','bs_l_did_mean', 'bs_l_did_min', 'bs_l_did_max']
    dev_lat=dev_lat.drop('did',axis=1)

    dev_lng =df.groupby(['did','messid']).agg(['mean', 'min', 'max'])[['bs_lng']].reset_index()
    dev_lng.columns = dev_lng.columns.droplevel()
    dev_lng.columns= ['did','messid','bs_L_did_mean', 'bs_L_did_min', 'bs_L_did_max']
    dev_lng=dev_lng.drop('did',axis=1)

    #On fait apparaitre une colonne contenant le messid pour la jointure

    df_feat = df_feat.reset_index()
    df_feat.rename(columns={'index':'messid'}, inplace=True)

    df_feat=df_feat.join(BCW_lat, on ='messid',how='left')
    df_feat=df_feat.join(BCW_lng, on ='messid',how='left')

    df_feat= pd.merge(df_feat,BC_lat,on='messid')
    df_feat= pd.merge(df_feat,BC_lng,on='messid')

    df_feat= pd.merge(df_feat,dev_lat,on='messid')
    df_feat= pd.merge(df_feat,dev_lng,on='messid')

    #On remet le messid dans l'index
    df_feat.set_index('messid',inplace=True)
    
    return df_feat, id_list

In [9]:
df_feat, id_list=feat_mat_const(X_train, listOfBs)

y_full = ground_truth_const(X_train, pos_train, id_list)
y_full.shape,df_feat.shape

((6068, 3), (6068, 273))

In [10]:
%%time
"""Cette celluel tourne pendant 2h !!!!!"""
"""Le résultat est stocké en pickle voir plus bas"""


#Code produit ensemble donc tu peux copier bêtement
Models={}
df_temp = df_feat[df_feat.columns[:-14]] #Ici on récupère la matrice df_feat sans columns supplémentaire
#Pour chaque base
for idx,base in enumerate(df_temp.columns):
    print(f"Base : {base}")
    
    #Prod de la sous matrice df_feat (lié à la base)
    df_feat_reduce=df_temp[df_temp[base]!=0]
    print(f"Nb message : {df_feat_reduce.shape[0]}")
    
    #On ne produit pas de modèle pour les bases non vu dans le train
    if df_feat_reduce.shape[0] :
        y=[];X=[]
        #On parcours tous les messages identifiés à une base
        for i,k in enumerate(df_feat_reduce.index):
            for j in df_temp.columns: #Je parcours toutes les bases et construit la matrice 259x259
                #print("tip")
                X.append(abs(df_temp.loc[k,j]-df_temp.loc[k,df_temp.columns].values))
                y.append(vincenty(base_coord.loc[j],msg_coord.loc[k]))
                #print("top")
        y=np.array(y)
        X=np.array(X)
        #On produit un modèle pour la base en question on note également l'indice de la base 
        model = xgb.XGBRegressor().fit(X,y);
        Models[base] = [idx,model]
        
        """ce bout de code ne sert QUE a vérifier si la méthode est bonne mais n'est pas nécessaire"""

        y_pred= Models[base][1].predict(X)
        #Calcul de le MAE
        sol=[]
        for j in range(y_pred.shape[0]//size):
            sol.append(y_pred[j*size:(j+1)*size][Models[base][0]])
        a =np.array(sol)
        b = X_train[X_train.bsid == base].dist.values 
        
        print(f"MAE de la base : {base} : {(np.abs(a-b)).mean()}")

    else :
        print(f"Base {base} en échec")

Base : 879
Nb message : 1
MAE de la base : 879 : 0.0030374271240241058
Base : 911
Nb message : 1
MAE de la base : 911 : 0.0012908613281261694
Base : 921
Nb message : 1
MAE de la base : 921 : 0.00010188110351805335
Base : 944
Nb message : 3
MAE de la base : 944 : 0.0005513832193990709
Base : 980
Nb message : 8
MAE de la base : 980 : 0.006038962215423438
Base : 1012
Nb message : 85
MAE de la base : 1012 : 0.22651944486512576
Base : 1086
Nb message : 67
MAE de la base : 1086 : 2.8489129506262025
Base : 1092
Nb message : 32
MAE de la base : 1092 : 1.2256982178645135
Base : 1120
Nb message : 1
MAE de la base : 1120 : 0.00043947338867411645
Base : 1131
Nb message : 0
Base 1131 en échec
Base : 1148
Nb message : 2
MAE de la base : 1148 : 0.0005229919433595853
Base : 1156
Nb message : 1
MAE de la base : 1156 : 166.88552357958986
Base : 1187
Nb message : 2
MAE de la base : 1187 : 70.96475746295167
Base : 1226
Nb message : 2
MAE de la base : 1226 : 2.526835102905274
Base : 1229
Nb message : 16
MA

MAE de la base : 3545 : 1.353017118405904
Base : 3546
Nb message : 22
MAE de la base : 3546 : 0.2574837415013747
Base : 3547
Nb message : 417
MAE de la base : 3547 : 1.3998327611196324
Base : 3548
Nb message : 570
MAE de la base : 3548 : 0.9757865115255088
Base : 3549
Nb message : 1
MAE de la base : 3549 : 70.62915384252929
Base : 3553
Nb message : 58
MAE de la base : 3553 : 1.2787806912234077
Base : 3554
Nb message : 129
MAE de la base : 3554 : 3.4355494994673617
Base : 3555
Nb message : 44
MAE de la base : 3555 : 0.5557488635976965
Base : 3556
Nb message : 472
MAE de la base : 3556 : 0.6172065785682838
Base : 3558
Nb message : 205
MAE de la base : 3558 : 1.1943030167541224
Base : 3559
Nb message : 165
MAE de la base : 3559 : 1.113034111408257
Base : 3562
Nb message : 866
MAE de la base : 3562 : 0.9178459159027192
Base : 3563
Nb message : 195
MAE de la base : 3563 : 1.7237384178297779
Base : 3565
Nb message : 265
MAE de la base : 3565 : 2.3742524304909183
Base : 3568
Nb message : 43
M

MAE de la base : 8451 : 1.257401573374348
Base : 8452
Nb message : 95
MAE de la base : 8452 : 1.2277945741795588
Base : 8453
Nb message : 49
MAE de la base : 8453 : 0.12588848970459918
Base : 8457
Nb message : 40
MAE de la base : 8457 : 0.1317774392385483
Base : 8470
Nb message : 481
MAE de la base : 8470 : 0.7717079154359338
Base : 8471
Nb message : 13
MAE de la base : 8471 : 1.190384750042255
Base : 8472
Nb message : 284
MAE de la base : 8472 : 1.2804369159983247
Base : 8473
Nb message : 112
MAE de la base : 8473 : 0.38380771483666554
Base : 8474
Nb message : 416
MAE de la base : 8474 : 0.8220770973647511
Base : 8475
Nb message : 322
MAE de la base : 8475 : 1.0439339584213863
Base : 8495
Nb message : 41
MAE de la base : 8495 : 0.42933625952734583
Base : 8509
Nb message : 156
MAE de la base : 8509 : 0.9093954502197047
Base : 8560
Nb message : 116
MAE de la base : 8560 : 1.9230704583782001
Base : 8746
Nb message : 74
MAE de la base : 8746 : 0.6330339003767066
Base : 8747
Nb message : 7

In [11]:
"""NE PAS RUNNER"""
with open (f"Models.pkl","wb") as fp :
    pickle.dump(Models, fp)

In [12]:
""" Runner cette cellule pour charger les Models"""
with open (f"Models.pkl","rb") as fp :
    Models = pickle.load(fp)

# Prédiction sur le Train pour qualifier la solution et le fit

In [13]:
df=X_train
df_feat, id_list=feat_mat_const(df, listOfBs)
df_temp = df_feat[df_feat.columns[:-14]]
df_group = df.groupby("messid", as_index=False)
nb_mess = len(df.messid.unique())
df_feat_new = pd.DataFrame(np.zeros((nb_mess, len(listOfBs))), columns=listOfBs, index=df.messid.unique())

In [14]:
%%time
for idx,base in enumerate(df_temp.columns):
    print(f"Base : {base}")
    #Prod de la sous matrice df_feat (lié à la base)
    df_feat_reduce=df_temp[df_temp[base]!=0]
    print(f"Nb message : {df_feat_reduce.shape[0]}")
    if df_feat_reduce.shape[0]:
        if Models.get(base,False) :
            X=[]
            #On parcours tous les messages identifié à une base
            for i,k in enumerate(df_feat_reduce.index):
                for j in df_temp.columns: #Je parcours toutes les bases et construit la matrice 259x259
                    X.append(abs(df_temp.loc[k,j]-df_temp.loc[k,df_temp.columns]).values)
            X=np.array(X)
            y= Models[base][1].predict(X)
            sol=[]
            for j in range(y.shape[0]//size):
                sol.append(y[j*size:(j+1)*size][Models[base][0]])
            df_feat_new.loc[df_feat_reduce.index.values,base]=np.array(sol)
        else:
            print(f"Base non modélisée {base}")
            """Ici on pourrait prévoire un amélioration qui une fois le message prédit on construit le modèle"""
    else :
        print(f"Base sans message {base}")

Base : 879
Nb message : 1
Base : 911
Nb message : 1
Base : 921
Nb message : 1
Base : 944
Nb message : 3
Base : 980
Nb message : 8
Base : 1012
Nb message : 85
Base : 1086
Nb message : 67
Base : 1092
Nb message : 32
Base : 1120
Nb message : 1
Base : 1131
Nb message : 0
Base sans message 1131
Base : 1148
Nb message : 2
Base : 1156
Nb message : 1
Base : 1187
Nb message : 2
Base : 1226
Nb message : 2
Base : 1229
Nb message : 16
Base : 1235
Nb message : 2
Base : 1237
Nb message : 1
Base : 1264
Nb message : 320
Base : 1266
Nb message : 3
Base : 1268
Nb message : 58
Base : 1292
Nb message : 6
Base : 1334
Nb message : 12
Base : 1344
Nb message : 184
Base : 1432
Nb message : 1
Base : 1443
Nb message : 1
Base : 1447
Nb message : 2
Base : 1463
Nb message : 2
Base : 1476
Nb message : 174
Base : 1526
Nb message : 11
Base : 1530
Nb message : 8
Base : 1534
Nb message : 1
Base : 1581
Nb message : 2
Base : 1594
Nb message : 290
Base : 1661
Nb message : 160
Base : 1730
Nb message : 1
Base : 1741
Nb messa

In [15]:
#Je rajoute le calcul des barycente que l'on trouve dans le df_feat
df_feat_new_TRAIN = pd.merge(df_feat_new,df_feat[df_feat.columns[-14:]],right_index=True,left_index=True) 

In [16]:
"""NE PAS RUNNER"""
with open (f"df_feat_new_TRAIN.pkl","wb") as fp :
    pickle.dump(df_feat_new_TRAIN, fp)

In [17]:
""" Runner cette cellule pour charger la df Feat du X_train"""
with open (f"df_feat_new_TRAIN.pkl","rb") as fp :
    df_feat_new_TRAIN = pickle.load(fp)

# Qualification avec les hypers paramètres trouvés

In [18]:
df_params = pd.read_csv("best_params.csv")

In [19]:
r1 = RandomForestRegressor(**get_hyperparameter('RandomForestRegressor', 'lng'))
r2 = GradientBoostingRegressor(**get_hyperparameter('GradientBoostingRegressor', 'lng'))
r3 = ExtraTreeRegressor(**get_hyperparameter('ExtraTreeRegressor', 'lng'))
r4 = xgb.XGBRegressor(**get_hyperparameter('XGBRegressor', 'lng'))
r5 = BaggingRegressor(**get_hyperparameter('BaggingRegressor', 'lng'))
Vr_lng = VotingRegressor(estimators=[('Et',r1),('Rf',r2),('Gb',r3),('Xg',r4),('Xdg',r5)])


r1 = RandomForestRegressor(**get_hyperparameter('RandomForestRegressor', 'lat'))
r2 = GradientBoostingRegressor(**get_hyperparameter('GradientBoostingRegressor', 'lat'))
r3 = ExtraTreeRegressor(**get_hyperparameter('ExtraTreeRegressor', 'lat'))
r4 = xgb.XGBRegressor(**get_hyperparameter('XGBRegressor', 'lat'))
r5 = BaggingRegressor(**get_hyperparameter('BaggingRegressor', 'lat'))
Vr_lat = VotingRegressor(estimators=[('Et',r1),('Rf',r2),('Gb',r3),('Xg',r4),('Xdg',r5)])

y_pred_lng = cross_val_predict(Vr_lng, df_feat_new_TRAIN, y_full.lng, cv=3)
y_pred_lat = cross_val_predict(Vr_lat, df_feat_new_TRAIN, y_full.lat, cv=3)

err_vec = Eval_geoloc(y_full.lat , y_full.lng, y_pred_lat, y_pred_lng)
np.percentile(err_vec, 80)

2.8407256000000003

# Prédiction du Test

In [20]:
df=X_test
df_feat, id_list=feat_mat_const(df, listOfBs)
df_temp = df_feat[df_feat.columns[:-14]]
df_group = df.groupby("messid", as_index=False)
nb_mess = len(df.messid.unique())
df_feat_new = pd.DataFrame(np.zeros((nb_mess, len(listOfBs))), columns=listOfBs, index=df.messid.unique())

In [21]:
%%time
for idx,base in enumerate(df_temp.columns):
    print(f"Base : {base}")
    #Prod de la sous matrice df_feat (lié à la base)
    df_feat_reduce=df_temp[df_temp[base]!=0]
    print(f"Nb message : {df_feat_reduce.shape[0]}")
    if df_feat_reduce.shape[0]:
        if Models.get(base,False) :
            X=[]
            #On parcours tous les messages identifié à une base
            for i,k in enumerate(df_feat_reduce.index):
                for j in df_temp.columns: #Je parcours toutes les bases et construit la matrice 259x259
                    X.append(abs(df_temp.loc[k,j]-df_temp.loc[k,df_temp.columns]).values)
            X=np.array(X)
            y= Models[base][1].predict(X)
            sol=[]
            for j in range(y.shape[0]//size):
                sol.append(y[j*size:(j+1)*size][Models[base][0]])
            df_feat_new.loc[df_feat_reduce.index.values,base]=np.array(sol)
        else:
            print(f"Base non modélisée {base}")
            """Ici on pourrait prévoire un amélioration qui une fois le message prédit on construit le modèle"""
    else :
        print(f"Base sans message {base}")

Base : 879
Nb message : 0
Base sans message 879
Base : 911
Nb message : 0
Base sans message 911
Base : 921
Nb message : 0
Base sans message 921
Base : 944
Nb message : 0
Base sans message 944
Base : 980
Nb message : 0
Base sans message 980
Base : 1012
Nb message : 80
Base : 1086
Nb message : 38
Base : 1092
Nb message : 2
Base : 1120
Nb message : 0
Base sans message 1120
Base : 1131
Nb message : 2
Base non modélisée 1131
Base : 1148
Nb message : 0
Base sans message 1148
Base : 1156
Nb message : 0
Base sans message 1156
Base : 1187
Nb message : 0
Base sans message 1187
Base : 1226
Nb message : 0
Base sans message 1226
Base : 1229
Nb message : 0
Base sans message 1229
Base : 1235
Nb message : 0
Base sans message 1235
Base : 1237
Nb message : 0
Base sans message 1237
Base : 1264
Nb message : 193
Base : 1266
Nb message : 0
Base sans message 1266
Base : 1268
Nb message : 12
Base : 1292
Nb message : 1
Base : 1334
Nb message : 2
Base : 1344
Nb message : 5
Base : 1432
Nb message : 1
Base : 1443

Base : 8437
Nb message : 462
Base : 8442
Nb message : 0
Base sans message 8442
Base : 8446
Nb message : 434
Base : 8449
Nb message : 25
Base : 8450
Nb message : 12
Base : 8451
Nb message : 30
Base : 8452
Nb message : 177
Base : 8453
Nb message : 3
Base : 8457
Nb message : 4
Base : 8470
Nb message : 116
Base : 8471
Nb message : 88
Base : 8472
Nb message : 453
Base : 8473
Nb message : 121
Base : 8474
Nb message : 437
Base : 8475
Nb message : 76
Base : 8495
Nb message : 0
Base sans message 8495
Base : 8509
Nb message : 72
Base : 8560
Nb message : 8
Base : 8746
Nb message : 107
Base : 8747
Nb message : 1
Base : 9783
Nb message : 22
Base non modélisée 9783
Base : 9784
Nb message : 0
Base sans message 9784
Base : 9899
Nb message : 397
Base : 9936
Nb message : 4
Base non modélisée 9936
Base : 9941
Nb message : 3
Base non modélisée 9941
Base : 9949
Nb message : 2
Base non modélisée 9949
Base : 10134
Nb message : 2
Base : 10148
Nb message : 0
Base sans message 10148
Base : 10151
Nb message : 29

In [22]:
df_feat_new_TEST = pd.merge(df_feat_new,df_feat[df_feat.columns[-14:]],right_index=True,left_index=True) 

In [23]:
"""NE PAS RUNNER"""
with open (f"df_feat_new_TEST.pkl","wb") as fp :
    pickle.dump(df_feat_new_TEST, fp)

In [24]:
""" Runner cette cellule pour charger la df Feat du X_test"""
with open (f"df_feat_new_TEST.pkl","rb") as fp :
    df_feat_new_TEST = pickle.load(fp)

In [25]:
r1 = RandomForestRegressor(**get_hyperparameter('RandomForestRegressor', 'lng'))
r2 = GradientBoostingRegressor(**get_hyperparameter('GradientBoostingRegressor', 'lng'))
r3 = ExtraTreeRegressor(**get_hyperparameter('ExtraTreeRegressor', 'lng'))
r4 = xgb.XGBRegressor(**get_hyperparameter('XGBRegressor', 'lng'))
r5 = BaggingRegressor(**get_hyperparameter('BaggingRegressor', 'lng'))
Vr_lng = VotingRegressor(estimators=[('Et',r1),('Rf',r2),('Gb',r3),('Xg',r4),('Xdg',r5)])


r1 = RandomForestRegressor(**get_hyperparameter('RandomForestRegressor', 'lat'))
r2 = GradientBoostingRegressor(**get_hyperparameter('GradientBoostingRegressor', 'lat'))
r3 = ExtraTreeRegressor(**get_hyperparameter('ExtraTreeRegressor', 'lat'))
r4 = xgb.XGBRegressor(**get_hyperparameter('XGBRegressor', 'lat'))
r5 = BaggingRegressor(**get_hyperparameter('BaggingRegressor', 'lat'))
Vr_lat = VotingRegressor(estimators=[('Et',r1),('Rf',r2),('Gb',r3),('Xg',r4),('Xdg',r5)])

Vr_lng.fit(df_feat_new_TRAIN, y_full.lng)
Vr_lat.fit(df_feat_new_TRAIN, y_full.lat)

y_pred_lng = Vr_lng.predict(df_feat_new_TEST)
y_pred_lat = Vr_lat.predict(df_feat_new_TEST)

#Il faut produire le fichier solution

In [26]:
y_pred_lng

array([-105.12001468, -105.08099715, -105.01428986, ..., -105.03130442,
       -105.02338449, -105.02420362])

In [27]:
y_pred_lat

array([39.72055927, 39.77837919, 39.6900661 , ..., 39.69068253,
       39.67845017, 39.68752195])

In [28]:
from ipyleaflet import Map, basemaps
msg_test = pd.DataFrame({'lat':y_pred_lat,'lng':y_pred_lng},index=df_feat_new_TEST.index)

barycentre = ((msg_test.lat.max()+msg_test.lat.min())/2,(msg_test.lng.min()+msg_test.lng.max())/2)

m = Map(center=barycentre, zoom=3, basemap = basemaps.OpenStreetMap.Mapnik)

message = Give_Marker_Cluster(msg_test)

m.add_layer(message)

m

OSError: [WinError 126] Le module spécifié est introuvable